# Newspaper classification

## Setup


Relevant libraries are imported:

In [1]:
from project_lib import Project
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from tensorflow import keras
from tensorflow.keras import backend as K

Data is loaded:

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
project = Project(project_id = project_id, project_access_token = project_token)

In [4]:
df_file = project.get_file("Newspaper_Data_Processed.csv")
df_file.seek(0)
df = pd.read_csv(df_file)

df.head()

,Title,Newspaper,DateTime,title_nlp_0,title_nlp_1,title_nlp_2,title_nlp_3,title_nlp_4,title_nlp_5,title_nlp_6,...,title_nlp_91,title_nlp_92,title_nlp_93,title_nlp_94,title_nlp_95,title_polarity,title_subjectivity,title_length,title_word_count,title_avg_word_length
0,Jetzt aber wirklich,SZ,2021-09-10 10:00:50.486499,-1.451339,-0.461309,1.613144,2.305896,3.325478,0.382818,-2.031413,...,0.426477,-0.126685,-2.633468,0.533935,2.881863,1.0,0.0,19,3,5.666667
1,Polizei setzt Pfefferspray und Schlagstöcke ge...,SZ,2021-09-10 10:00:50.486499,-0.181093,0.741531,0.245324,-0.954865,-0.580439,-0.799379,-1.523495,...,0.118394,0.913981,-0.542663,0.803660,-0.032087,0.0,0.0,68,8,7.625000
2,Reservisten sollen rechtsextreme Wehrsportgrup...,SZ,2021-09-10 10:00:50.486499,0.605318,-0.240080,0.321833,-1.108226,-1.045246,2.174628,0.767455,...,0.265194,-0.731849,-0.401369,0.108651,-0.963425,0.0,0.0,63,6,9.666667
3,Typisch Einzelkind - oder?,SZ,2021-09-10 10:00:50.486499,1.387215,-1.893740,0.447758,0.436939,2.100209,-0.737141,-0.660538,...,0.133348,-0.452347,-0.411469,1.196424,0.763209,0.0,0.0,26,4,5.750000
4,Und jetzt die volle Ladung Djokovic,SZ,2021-09-10 10:00:50.486499,0.051133,-0.446347,1.168035,0.361419,2.637187,0.526604,-0.706850,...,1.040411,-0.189703,-0.743807,-0.363708,-0.794379,0.0,0.0,35,6,5.000000


The shape of the data is: 

In [5]:
df.shape

(2541, 104)

For each newspaper, the following number of title entries is available:

In [6]:
df[['Newspaper', 'Title']].groupby('Newspaper').count()

,Title
Newspaper,
SZ,960
Welt,1581


Classification-relevant variables are defined:

In [7]:
number_newspapers = df['Newspaper'].nunique()

feature_columns = [col for col in df.columns if 'title_nlp_' in col]
feature_columns.extend(['title_polarity', 'title_subjectivity'])
feature_columns.extend(['title_length', 'title_word_count', 'title_avg_word_length'])

## Training & test split

Data is split into training and test sets using 80% for training and 20% for testing:

In [8]:
n_samples, n_features = df.shape[0], len(feature_columns)
rng = np.random.RandomState(0)

X = np.array(df[feature_columns])
y = np.array(df['Newspaper'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=rng, test_size = 0.2)

## Feature scaling

Features are scaled for classification using the *MinMaxScaler()* implemented in sklearn:

In [9]:
min_max_scaler = preprocessing.MinMaxScaler()

X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.transform(X_test)

## Classification

Classification is run using three different algorithms:

1. Linear Support Vector Machine (SVM) is used for computationally simple proof-of-concept prediction.
2. XGBoost is ussed as a state-of-the-art algorithm for supervised machine learning.
3. A neural network is used as state-of-the-art deep learning model for comparison with XGBoost and SVM algorithms.

Both XGBoost and the neural network were selected as they can model non-linear associations. This is likely to be important for complex text-based features that include combinations of different words and sentiment.

As **evaluation metric**, the primary evaluation metric of interest is the F1-score as it is more robust against imbalanced class data (as is the case here). However, the standard model accuracy is also provided as comparison.

### Support vector classifier

In [10]:
svc_parameters = {'kernel':('linear', 'rbf'), 'C':[0.1,1,8,16,32]}
svc = svm.SVC()
clf_svm = GridSearchCV(svc, svc_parameters, cv=5)
clf_svm.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 8, 16, 32], 'kernel': ('linear', 'rbf')})

In [11]:
y_pred = clf_svm.predict(X_test)

print("The accuracy for the test set is: " + str(accuracy_score(y_test, y_pred)))
print("The F1 score for the test set is: " + str(f1_score(y_test, y_pred, average = 'weighted')))

The accuracy for the test set is: 0.9724950884086444
The F1 score for the test set is: 0.9724077011734858


### XGBoost

In [12]:
xgboost_parameters = {
    "learning_rate": [0.01, 0.1, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 3),
    "min_samples_leaf": np.linspace(0.1, 0.5, 3),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "subsample":[0.5, 0.8, 1.0],
    "n_estimators":[10]
    }

xgboost = GradientBoostingClassifier(random_state=0)
clf_xgboost = GridSearchCV(xgboost, xgboost_parameters, cv=5)
clf_xgboost.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=0),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 8],
                         'max_features': ['log2', 'sqrt'],
                         'min_samples_leaf': array([0.1, 0.3, 0.5]),
                         'min_samples_split': array([0.1, 0.3, 0.5]),
                         'n_estimators': [10], 'subsample': [0.5, 0.8, 1.0]})

In [13]:
y_pred = clf_xgboost.predict(X_test)

print("The accuracy for the test set is: " + str(accuracy_score(y_test, y_pred)))
print("The F1 score for the test set is: " + str(f1_score(y_test, y_pred, average = 'weighted')))

The accuracy for the test set is: 0.8703339882121808
The F1 score for the test set is: 0.8684084129266718


### Neural network

One-hot encoded vectors are created for the newspaper outcome variable to align with *keras* format:

In [14]:
y_train_cat = pd.get_dummies(y_train)
y_test_cat = pd.get_dummies(y_test)

To assess the F1-score for the Keras model, this metric needs to be defined manually as it is not implemented in *keras*. The functions are therefore defined as indicated on https://neptune.ai/blog/implementing-the-macro-f1-score-in-keras:

In [15]:
# F1 measures definition according to: F1 = 2 * (precision * recall) / (precision + recall)
def custom_f1(y_true, y_pred):    
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        
        recall = TP / (Positives+K.epsilon())    
        return recall 
    
    
    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
        precision = TP / (Pred_Positives+K.epsilon())
        return precision 
    
    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

The neural network is built using three dense layers with relu activation function and a variable amount of 32-64 nodes. A dropout layer with 0.5 dropout is defined to enhance model generalisability:

In [16]:
model = keras.Sequential([
    keras.layers.Dense(32, input_shape=(X_train.shape[1],), activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(2, activation='sigmoid')])

model.compile(optimizer='adam', 
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[custom_f1,'accuracy'])

model.fit(X_train, y_train_cat, batch_size=4, epochs=10)

Epoch 1/10
508/508 [==============================] - 3s 4ms/step - loss: 0.6402 - custom_f1: 0.6445 - accuracy: 0.6583
Epoch 2/10
508/508 [==============================] - 2s 3ms/step - loss: 0.3077 - custom_f1: 0.8715 - accuracy: 0.8797
Epoch 3/10
508/508 [==============================] - 2s 3ms/step - loss: 0.2924 - custom_f1: 0.8839 - accuracy: 0.8866
Epoch 4/10
508/508 [==============================] - 2s 3ms/step - loss: 0.2184 - custom_f1: 0.9138 - accuracy: 0.9145
Epoch 5/10
508/508 [==============================] - 2s 3ms/step - loss: 0.1689 - custom_f1: 0.9293 - accuracy: 0.9306
Epoch 6/10
508/508 [==============================] - 2s 3ms/step - loss: 0.1512 - custom_f1: 0.9487 - accuracy: 0.9485
Epoch 7/10
508/508 [==============================] - 2s 3ms/step - loss: 0.1634 - custom_f1: 0.9386 - accuracy: 0.9395
Epoch 8/10
508/508 [==============================] - 2s 3ms/step - loss: 0.1400 - custom_f1: 0.9529 - accuracy: 0.9523
Epoch 9/10
508/508 [====================

The model is evaluated on the test set:

In [17]:
model.evaluate(X_test, y_test_cat)

16/16 [==============================] - 0s 2ms/step - loss: 0.1498 - custom_f1: 0.9373 - accuracy: 0.9371


[0.14981158077716827, 0.9372979402542114, 0.9371316432952881]

## Conclusion

Classification of newspaper title data shows that it is possible to classify SZ and Welt newspapers from title features alone with relatively high accuracy. For prediction using data across multiple days, best prediction is achieved using SVC (F1 score=*0.97*), followed by somewhat worse prediction using neural network classification (F1 score=*0.94*) and similar performance for the XGBoost classifier (F1 score=*0.87*).